# Portfolio Optimisation

In this notebook we will be walking through an example of constructing an Efficient Frontier, a key part of Modern Portfolio Theory!

## Part 1: Importing the libraries and data

In [37]:
#Libraries for data preprocessing and analysis

import numpy as np
import pandas as pd
import yfinance as yf

#Libraries for graph plotting

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

In [38]:
## Data ##

tickers = 'MSFT AAPL AMZN NVDA GOOGL META JPM PG JNJ ADBE' #Tickers so yfinance knows what stocks to get
df = yf.download(tickers = tickers, start = '2013-01-01', end = '2023-01-01', interval = '1d') #Parameters for yfinance
df = df.xs('Adj Close', axis = 1) #Select only the adjusted close values for each stock

daily_returns = df.apply(lambda x:x.pct_change()).fillna(0) #Calculate the daily percentage change of each stock and put this in a new df
daily_returns


[*********************100%***********************]  10 of 10 completed


Ticker,AAPL,ADBE,AMZN,GOOGL,JNJ,JPM,META,MSFT,NVDA,PG
Date,,,,,,,,,,
2013-01-02 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-01-03 00:00:00+00:00,-0.012623,-0.015389,0.004547,0.000581,-0.001412,-0.002015,-0.008214,-0.013396,0.000787,-0.006341
2013-01-04 00:00:00+00:00,-0.027855,0.010066,0.002592,0.019760,0.011450,0.017725,0.035650,-0.018716,0.032993,0.002030
2013-01-07 00:00:00+00:00,-0.005882,-0.004983,0.035925,-0.004363,-0.002096,0.001102,0.022949,-0.001869,-0.028898,-0.006803
2013-01-08 00:00:00+00:00,0.002692,0.005272,-0.007748,-0.001973,0.000140,0.001982,-0.012237,-0.005246,-0.021927,-0.001603
...,...,...,...,...,...,...,...,...,...,...
2022-12-23 00:00:00+00:00,-0.002798,0.005735,0.017425,0.016750,0.002542,0.004745,0.007855,0.002267,-0.008671,0.002825
2022-12-27 00:00:00+00:00,-0.013878,-0.009928,-0.025924,-0.020621,-0.000282,0.003504,-0.009827,-0.007414,-0.071353,0.008715
2022-12-28 00:00:00+00:00,-0.030685,-0.020174,-0.014692,-0.015677,-0.004340,0.005465,-0.010780,-0.010255,-0.006019,-0.012926


In [39]:
## Annualised Returns ##

ann_return = (1+daily_returns.mean())**252 -1 #Why do we -1 from this data? Why is is multiplied by 252
ann_return

#Output is annual returns for each stock!

Ticker
AAPL     0.279668
ADBE     0.306158
AMZN     0.272166
GOOGL    0.215253
JNJ      0.143774
JPM      0.188778
META     0.244950
MSFT     0.312727
NVDA     0.629256
PG       0.132318
dtype: float64

In [40]:
## Annualised Portfolio Variance (variance & covariance) ##

ann_cov = daily_returns.cov()*252
ann_cov

#Output is a correlation matrix
#Notice anything about this matrix?

Ticker,AAPL,ADBE,AMZN,GOOGL,JNJ,JPM,META,MSFT,NVDA,PG
Ticker,,,,,,,,,,
AAPL,0.084430,0.050761,0.047987,0.045097,0.018318,0.032769,0.053265,0.049309,0.069096,0.019514
ADBE,0.050761,0.099350,0.059018,0.053138,0.019584,0.032383,0.062971,0.058668,0.081095,0.019289
AMZN,0.047987,0.059018,0.106305,0.055505,0.015881,0.027444,0.067454,0.052525,0.072842,0.014632
GOOGL,0.045097,0.053138,0.055505,0.073275,0.018238,0.032475,0.061596,0.051020,0.065947,0.017667
JNJ,0.018318,0.019584,0.015881,0.018238,0.031244,0.020373,0.017145,0.020378,0.019967,0.018223
JPM,0.032769,0.032383,0.027444,0.032475,0.020373,0.071830,0.033097,0.034275,0.043811,0.018278
META,0.053265,0.062971,0.067454,0.061596,0.017145,0.033097,0.145174,0.052606,0.077249,0.015854
MSFT,0.049309,0.058668,0.052525,0.051020,0.020378,0.034275,0.052606,0.073055,0.070842,0.021503
NVDA,0.069096,0.081095,0.072842,0.065947,0.019967,0.043811,0.077249,0.070842,0.195935,0.021357


## Part 2: Creating the data to plot


Now we have our annualised returns and variance data, we can create variables useful in constructing our graph

In [41]:
# For simplicity, and in accordance with Ben's points we are taking a Monte Carlo approach
# We will create portfolios of random assets with random weightings and plot them in mean-variance space

In [42]:
## Set the amount of assets we want in our portfolio, and how many different combinations we want on our graph ##

n_assets = 5 #Must be less than or equal to number of assets in our portfolio
n_portfolios = 1000 #Number of portfolios being created
np.random.seed(75) #For the random weight generation


Link for more information the equations used:

https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie

In [43]:
## Loop to generate portfolios, creating mean expected return and variance for each ##

asset_list = [] #What might these lists be for?
weights_list = []
mean = []
variance = []

for i in range(n_portfolios):

    assets = np.random.choice(list(daily_returns.columns), n_assets, replace = False)
    #Generates list of assets, replace = False ensures no duplicate assets

    asset_list.append(assets)

    weights = np.random.rand(n_assets)
    weights = (weights / sum(weights)).round(4) #2dp for percentage weighting

    weights_list.append(weights)


    port_exp_ret = 0
    port_exp_var = 0
    #Resets the return and variance of the portfolio for each iteration

    for i in range(len(assets)):
        port_exp_ret += weights[i] * ann_return.loc[assets[i]]
        #Iterating through each asset in the portfolio and adding the assets weighted return to the overall portfolio return

        for j in range(len(assets)):
            port_exp_var += weights[i] * weights[j] * ann_cov.loc[assets[i], assets[j]]
            #General formula for portfolio variance which sums the variance of asset(i) with all other assets in the portfolio, for every asset!

    mean.append(port_exp_ret)
    variance.append(port_exp_var)

In [44]:
#For convenience and better analysis, we can create a dataframe with all our values - these are what all the lists are for!

plot_data = pd.DataFrame(index = range(n_portfolios),data = zip(mean, variance, asset_list, weights_list),
                         columns = ['Mean', 'Variance', 'Asset List', 'Asset Weights']) #Creating our dataframe and passing in all our lists

plot_data['Sharpe Ratio'] = (plot_data['Mean'] - 0.05) / plot_data['Variance']**0.5 #Add a column for our sharpe ratio
#What might 0.05 mean here?

plot_data

,Mean,Variance,Asset List,Asset Weights,Sharpe Ratio
0,0.251793,0.049321,"[META, JNJ, MSFT, AMZN, JPM]","[0.2441, 0.2225, 0.417, 0.0915, 0.0249]",0.908634
1,0.269675,0.052870,"[ADBE, GOOGL, JPM, MSFT, AAPL]","[0.2259, 0.1458, 0.1676, 0.2616, 0.1991]",0.955377
2,0.298136,0.042913,"[JNJ, AAPL, PG, JPM, NVDA]","[0.1501, 0.1796, 0.2855, 0.1218, 0.2631]",1.197824
3,0.307744,0.053242,"[AMZN, NVDA, JPM, GOOGL, JNJ]","[0.2769, 0.23, 0.1732, 0.1256, 0.1942]",1.117025
4,0.257172,0.048096,"[AAPL, NVDA, JPM, AMZN, JNJ]","[0.1985, 0.0947, 0.4714, 0.1498, 0.0856]",0.944659
...,...,...,...,...,...
995,0.342454,0.073403,"[NVDA, GOOGL, MSFT, META, ADBE]","[0.1951, 0.17, 0.2992, 0.2165, 0.1192]",1.079448
996,0.187063,0.028292,"[PG, GOOGL, AAPL, JNJ, NVDA]","[0.1955, 0.2671, 0.1113, 0.4028, 0.0233]",0.814868
997,0.204497,0.038649,"[JNJ, GOOGL, PG, ADBE, MSFT]","[0.0101, 0.382, 0.3765, 0.2124, 0.0191]",0.785869
998,0.209515,0.031905,"[JNJ, GOOGL, PG, MSFT, AAPL]","[0.2556, 0.1787, 0.2017, 0.1767, 0.1872]",0.893039


In [45]:
len(plot_data) #Check the length of our dataframe

1000

## Part 3:Plotting the Efficient Frontier graph


Using plotly we can make excellent interactive graphs to display our different portfolios

In [46]:
fig = go.Figure() #Creates an instance of go (graph object) similiar to figure = plt.figure

fig.add_trace(go.Scatter(x=plot_data['Variance']**0.5, y=plot_data['Mean'], #Note our x axis becomes standard deviation so is square rooted
                      marker=dict(color=plot_data['Sharpe Ratio'], #Mark each point by its sharpe ratio
                                  showscale=True, #Parameters for the marker
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ),
                      mode='markers'))

fig.update_layout(template='plotly_white', #Generic formatting
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)

fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': array([0.9086338 , 0.95537741, 1.1978242 , ..., 0.78586878, 0.89303947,
                                         0.87349019]),
                         'colorbar': {'title': {'text': 'Sharpe<br>Ratio'}},
                         'colorscale': [[0.0, 'rgb(103,0,31)'], [0.1,
                                        'rgb(178,24,43)'], [0.2, 'rgb(214,96,77)'],
                                        [0.3, 'rgb(244,165,130)'], [0.4,
                                        'rgb(253,219,199)'], [0.5,
                                        'rgb(247,247,247)'], [0.6,
                                        'rgb(209,229,240)'], [0.7,
                                        'rgb(146,197,222)'], [0.8,
                                        'rgb(67,147,195)'], [0.9,
                                        'rgb(33,102,172)'], [1.0, 'rgb(5,48,97)']],
                         'line': {'width': 1},
                         'showscale': True,
                         'size': 7},
              'mode': 'markers',
              'type': 'scatter',
              'x': array([0.22208366, 0.22993551, 0.20715559, ..., 0.1965944 , 0.1786208 ,
                          0.18857273]),
              'y': array([0.25179272, 0.26967519, 0.29813597, ..., 0.2044974 , 0.20951543,
                          0.21471643])}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Sharpe Ratio'}}},
               'height': 500,
               'template': '...',
               'title': {'text': 'Sample of Random Portfolios'},
               'width': 850,
               'xaxis': {'title': {'text': 'Annualised Risk (Volatility)'}},
               'yaxis': {'title': {'text': 'Annualised Return'}}}
})

# Data Analysis on our Portfolios

By utilising the versatility of Pandas and DataFrames, we can extract meaningful insights on our different portfolios

In [47]:
# We can search for max values from our dataframe

plot_data['Sharpe Ratio'].idxmax() #Locates the row with the maximum sharpe ratio

396

In [48]:
# What if we want to know more about our portfolio with the highest sharpe ratio?

plot_data.iloc[plot_data['Sharpe Ratio'].idxmax()] #Outputs the related info


Mean                                            0.364243
Variance                                        0.059112
Asset List                   [AAPL, AMZN, JNJ, NVDA, PG]
Asset Weights    [0.1201, 0.0499, 0.3916, 0.408, 0.0305]
Sharpe Ratio                                    1.292488
Name: 396, dtype: object

In [49]:
# We can also search for portfolios with risk below a certain level, and then order them by which metric we want!

plot_data[plot_data['Variance']**0.5 < 0.3].sort_values('Sharpe Ratio', axis =0, ascending = False) #Square rooting our variance

#Here we are ordering our portfolios by the best sharpe ratio at a volatility (standard deviation) of less than 30%

,Mean,Variance,Asset List,Asset Weights,Sharpe Ratio
396,0.364243,0.059112,"[AAPL, AMZN, JNJ, NVDA, PG]","[0.1201, 0.0499, 0.3916, 0.408, 0.0305]",1.292488
654,0.380217,0.066277,"[ADBE, JNJ, AAPL, PG, NVDA]","[0.0842, 0.0388, 0.0651, 0.3627, 0.4492]",1.282675
933,0.353131,0.056183,"[ADBE, MSFT, GOOGL, NVDA, JNJ]","[0.1262, 0.143, 0.0167, 0.3368, 0.3773]",1.278871
24,0.344825,0.053263,"[JNJ, NVDA, AAPL, MSFT, GOOGL]","[0.4755, 0.3725, 0.0503, 0.0624, 0.0394]",1.277469
917,0.360002,0.059485,"[JNJ, MSFT, PG, GOOGL, NVDA]","[0.3278, 0.1007, 0.0386, 0.1428, 0.3902]",1.271038
...,...,...,...,...,...
334,0.197404,0.049281,"[JNJ, AAPL, GOOGL, META, PG]","[0.058, 0.0047, 0.2552, 0.3779, 0.3042]",0.663998
868,0.219489,0.065268,"[ADBE, MSFT, NVDA, JNJ, JPM]","[0.0271, 0.0262, 0.0557, 0.0056, 0.8854]",0.663421
564,0.215123,0.062941,"[META, JPM, JNJ, GOOGL, PG]","[0.3188, 0.0157, 0.068, 0.5454, 0.0521]",0.658174
46,0.168916,0.033775,"[MSFT, GOOGL, PG, JNJ, JPM]","[0.0335, 0.0008, 0.2168, 0.2625, 0.4865]",0.647054


## Extra: Improving the graph and adding features

We are now adding a dominating condition to remove any unwanted portfolios

In [50]:
# Theres are lots of portfolios below the efficient frontier, so we may as well get rid of these using something called domination
# If two portfolios have equal return, we will chose the one with less risk, and similiarly at the same risk level we want the most return

In [51]:
# Same code as before, but the variables have an _2 so previous lines of code can be ran/ messed around with without variables issues

n_assets_2 = 5
n_portfolios_2 = 10000 #10000 portfolios this time

asset_list_2 = []
weights_list_2 = []
mean_2 = []
variance_2 = []

for i in range(n_portfolios_2):

    assets = np.random.choice(list(daily_returns.columns), n_assets_2, replace = False)
    asset_list_2.append(assets)

    weights = np.random.rand(n_assets_2)
    weights = (weights / sum(weights)).round(4)
    weights_list_2.append(weights)


    port_exp_ret = 0
    port_exp_var = 0

    for i in range(len(assets)):
        port_exp_ret += weights[i] * ann_return.loc[assets[i]]

        for j in range(len(assets)):
            port_exp_var += weights[i] * weights[j] * ann_cov.loc[assets[i], assets[j]]

    mean_2.append(port_exp_ret)
    variance_2.append(port_exp_var)

In [52]:
# Same as before we create our dataframe

plot_data_2 = pd.DataFrame(index = range(n_portfolios_2),data = zip(mean_2, variance_2, asset_list_2, weights_list_2), columns = ['Mean', 'Variance', 'Asset List', 'Asset Weights'])
plot_data_2['Sharpe Ratio'] = (plot_data_2['Mean'] - 0.05) / plot_data_2['Variance']**0.5
plot_data_2

,Mean,Variance,Asset List,Asset Weights,Sharpe Ratio
0,0.241502,0.048106,"[JPM, MSFT, GOOGL, ADBE, PG]","[0.3074, 0.1587, 0.2154, 0.2607, 0.0579]",0.873115
1,0.244309,0.066099,"[PG, MSFT, META, ADBE, GOOGL]","[0.0864, 0.0136, 0.3295, 0.276, 0.2946]",0.755779
2,0.387399,0.084281,"[NVDA, PG, META, ADBE, AMZN]","[0.3164, 0.0477, 0.0689, 0.3176, 0.2494]",1.162191
3,0.258942,0.040957,"[JNJ, JPM, GOOGL, AAPL, NVDA]","[0.3445, 0.0327, 0.307, 0.1761, 0.1397]",1.032437
4,0.239375,0.041104,"[AAPL, AMZN, JNJ, JPM, MSFT]","[0.2825, 0.0043, 0.1871, 0.26, 0.2661]",0.934079
...,...,...,...,...,...
9995,0.319625,0.051330,"[ADBE, PG, MSFT, NVDA, JPM]","[0.0862, 0.2053, 0.2148, 0.24, 0.2536]",1.190075
9996,0.250226,0.067410,"[ADBE, JNJ, JPM, AAPL, META]","[0.1746, 0.1178, 0.0281, 0.2329, 0.4466]",0.771181
9997,0.317316,0.067667,"[AMZN, META, JPM, NVDA, GOOGL]","[0.2157, 0.1785, 0.1881, 0.2161, 0.2016]",1.027633
9998,0.267573,0.065175,"[JNJ, NVDA, META, AMZN, JPM]","[0.0864, 0.076, 0.2932, 0.3926, 0.1518]",0.852245


In [53]:
# Dominating condition to filter out our undesirable portfolios

mean_var_pairs = plot_data_2[['Mean', 'Variance']].to_numpy() #Changing our mean and variance into an array to be interated through
dominant_portfolio = [] #Empty array for our desirable portfolios

for R_check,V_check in mean_var_pairs: #R_check and V_check is now the return and var and loops through each item in in our data

    should_append = True   #Initial condition is that the portfolio from mean_variance_pairs is a dominant portfolio

    for R,V in mean_var_pairs: #Second iteration within the first loop which compares the R_check and V_check against every other portfolio created
        if (R > R_check) & (V < V_check): #If the portfolio is dominated by any other, break out the loop and dont add it to dominant pairs array

            should_append = False
            break

    if should_append == True:
        dominant_portfolio.append([R_check, V_check])   #Otherwise add it to the dominant pairs list

#This has two loops and compares one mean_variance pair against all others
#If R_check/V_check portfolio isnt dominated by any other, it is a dominant portfolio
#Then iterates through the whole mean_variance pairs array


In [54]:
#Now we are creating our new dataframe through a merge with the original dataframe, on our dominant portfolios

dominant_df = pd.DataFrame(index = range(len(dominant_portfolio)), data = dominant_portfolio, columns = ['Mean', 'Variance'])

dominant_df = pd.merge(dominant_df, plot_data_2, how = 'left') #Merged on the dominant portfolios, meaning only dominant values are carried over

dominant_df

,Mean,Variance,Asset List,Asset Weights,Sharpe Ratio
0,0.272066,0.036113,"[JNJ, MSFT, NVDA, AAPL, PG]","[0.3565, 0.2047, 0.1653, 0.1125, 0.1611]",1.168557
1,0.208979,0.027490,"[AMZN, PG, ADBE, NVDA, JNJ]","[0.0091, 0.2938, 0.0924, 0.1079, 0.4969]",0.958844
2,0.349211,0.055380,"[JNJ, ADBE, MSFT, NVDA, META]","[0.3661, 0.1229, 0.1639, 0.3192, 0.0279]",1.271458
3,0.421304,0.080943,"[GOOGL, JNJ, PG, AAPL, NVDA]","[0.0801, 0.2771, 0.0201, 0.0866, 0.5361]",1.305092
4,0.333788,0.049376,"[MSFT, NVDA, PG, JPM, JNJ]","[0.0967, 0.3541, 0.1604, 0.0801, 0.3087]",1.277127
...,...,...,...,...,...
95,0.418749,0.080236,"[JPM, JNJ, NVDA, MSFT, AAPL]","[0.0096, 0.0843, 0.4016, 0.3332, 0.1713]",1.301805
96,0.395655,0.071782,"[NVDA, MSFT, AMZN, PG, JPM]","[0.4299, 0.1592, 0.1045, 0.1936, 0.1128]",1.290136
97,0.263320,0.034053,"[MSFT, PG, NVDA, AMZN, JNJ]","[0.1623, 0.271, 0.1792, 0.064, 0.3236]",1.155988
98,0.352253,0.055918,"[JPM, AMZN, JNJ, NVDA, META]","[0.0379, 0.0517, 0.4904, 0.4102, 0.0098]",1.278188


In [55]:
# There are less portfolios now!

len(dominant_df)

100

In [56]:
#Same code as before

fig = go.Figure()

fig.add_trace(go.Scatter(x=dominant_df['Variance']**0.5, y=dominant_df['Mean'],
                      marker=dict(color=dominant_df['Sharpe Ratio'],
                                  showscale=True,
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ),
                      mode='markers'))

fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Dominant Portfolios',
                  width=850,
                  height=500)

fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': array([1.1685571 , 0.95884435, 1.2714581 , 1.30509237, 1.27712672, 1.00381308,
                                         1.16247231, 1.27765747, 1.26234267, 1.31175153, 1.25997581, 1.28310285,
                                         1.29509424, 1.3220972 , 1.2653897 , 1.17226964, 1.31123052, 1.11542971,
                                         1.31123453, 1.28147625, 1.24042236, 1.28532913, 1.20502914, 1.30396479,
                                         1.24407545, 1.30525374, 0.94422297, 1.27748265, 1.2748431 , 1.29207821,
                                         1.29540007, 1.12154839, 1.31470412, 1.20531853, 1.24946309, 1.242134  ,
                                         1.26966549, 1.31132782, 1.24543965, 1.31244929, 1.20093606, 1.26516623,
                                         1.11322765, 1.30860466, 1.19062222, 1.26202211, 1.28851637, 1.22517747,
                                         0.98164166, 1.00987115, 1.05040885, 0.91830206, 1.26299592, 1.29517116,
                                         1.2704186 , 1.24209276, 1.16704037, 1.31708153, 1.13292636, 1.28674379,
                                         1.20179693, 1.18212012, 0.76403689, 1.2829403 , 1.29066802, 1.28715507,
                                         1.3216103 , 1.2870751 , 1.27809576, 1.29656265, 1.30342672, 1.27291418,
                                         1.23921502, 1.17466749, 1.25116728, 1.17937875, 1.15692927, 1.23044813,
                                         1.19630102, 0.80982402, 1.31578993, 1.31138796, 1.28543232, 1.30790582,
                                         1.2990664 , 1.29769974, 1.04551657, 1.29046346, 0.72802202, 1.20928631,
                                         1.25200061, 0.70482063, 0.8368164 , 1.28316042, 0.93416996, 1.30180545,
                                         1.29013556, 1.15598784, 1.27818824, 1.30647613]),
                         'colorbar': {'title': {'text': 'Sharpe<br>Ratio'}},
                         'colorscale': [[0.0, 'rgb(103,0,31)'], [0.1,
                                        'rgb(178,24,43)'], [0.2, 'rgb(214,96,77)'],
                                        [0.3, 'rgb(244,165,130)'], [0.4,
                                        'rgb(253,219,199)'], [0.5,
                                        'rgb(247,247,247)'], [0.6,
                                        'rgb(209,229,240)'], [0.7,
                                        'rgb(146,197,222)'], [0.8,
                                        'rgb(67,147,195)'], [0.9,
                                        'rgb(33,102,172)'], [1.0, 'rgb(5,48,97)']],
                         'line': {'width': 1},
                         'showscale': True,
                         'size': 7},
              'mode': 'markers',
              'type': 'scatter',
              'x': array([0.19003399, 0.16580224, 0.23532885, 0.28450424, 0.22220821, 0.17163812,
                          0.19316408, 0.23408656, 0.22767792, 0.32614955, 0.21887628, 0.32572435,
                          0.34989237, 0.38981137, 0.22621879, 0.18707665, 0.30086239, 0.18316721,
                          0.29079951, 0.23122634, 0.21123757, 0.35857142, 0.20344038, 0.2773359 ,
                          0.20450443, 0.33297415, 0.163953  , 0.21967497, 0.23407109, 0.27517272,
                          0.2758452 , 0.17968453, 0.36129775, 0.19880856, 0.21326482, 0.21037296,
                          0.32026542, 0.24998867, 0.21462603, 0.27055648, 0.19719271, 0.22649294,
                          0.17894789, 0.28761286, 0.20273096, 0.21549883, 0.24307086, 0.20907334,
                          0.16815518, 0.16967916, 0.17351052, 0.16170231, 0.22938748, 0.25907935,
                          0.22956336, 0.20585373, 0.18861516, 0.3793422 , 0.18445661, 0.23714902,
                          0.20331954, 0.19361036, 0.15787553, 0.31670189, 0.26059726, 0.29637265,
                          0.30218795, 0.26578725, 0.24218314, 0.25564545, 0.25418985, 0.23990377,


# Coding challenge!

The task for you guys is to, using the the given data set below, find the top 5 portfolios with the highest sharpe ratio for 0.3 volatility, and then the portfolio with the best sharpe ratio overall - you can also try and plot an Efficient Frontier graph!

Hint:
- Pandas has really good documentation online on dataframe conditional filtering
- .idxmax and .loc (!)

In [57]:
#Getting our data
chall_tickers = 'AAPL MSFT AMZN NVDA META AVGO GOOGL GOOG TSLA ADBE COST PEP CSCO NFLX AMD'
chall_df =  yf.download(tickers = chall_tickers, start = '2013-01-01', end = '2023-01-01', interval = '1d')
chall_df = chall_df.xs('Adj Close', axis = 1)

#Getting our daily returns
chall_daily_returns = chall_df.apply(lambda x:x.pct_change()).fillna(0)

#Getting our mean and variance data
chall_ann_return = (1+chall_daily_returns.mean())**252 -1
chall_ann_cov = chall_daily_returns.cov()*252

#Create some parameters for generating our random portfolios
chall_n_assets = 5
chall_n_portfolios = 1000
np.random.seed(75)

[*********************100%***********************]  15 of 15 completed


In [58]:
#Creating our lists
chall_asset_list = []
chall_weights_list = []
chall_mean = []
chall_variance = []

#Creating our random portfolios
for i in range(chall_n_portfolios):

    assets = np.random.choice(list(chall_daily_returns.columns), chall_n_assets, replace = False)
    chall_asset_list.append(assets)
    weights = np.random.rand(chall_n_assets)
    weights = (weights / sum(weights)).round(4)
    chall_weights_list.append(weights)

    port_exp_ret = 0
    port_exp_var = 0

    for i in range(len(assets)):
        port_exp_ret += weights[i] * chall_ann_return.loc[assets[i]]
        for j in range(len(assets)):
            port_exp_var += weights[i] * weights[j] * chall_ann_cov.loc[assets[i], assets[j]]
    chall_mean.append(port_exp_ret)
    chall_variance.append(port_exp_var)

In [59]:
#Creating our dataframe for our portfolios
chall_data = pd.DataFrame(index = range(chall_n_portfolios),data = zip(chall_mean, chall_variance, chall_asset_list, chall_weights_list),
                         columns = ['Mean', 'Variance', 'Asset List', 'Asset Weights'])
chall_data['Sharpe Ratio'] = (chall_data['Mean'] - 0.05) / chall_data['Variance']**0.5

chall_data

,Mean,Variance,Asset List,Asset Weights,Sharpe Ratio
0,0.365044,0.058847,"[COST, GOOG, AVGO, MSFT, NVDA]","[0.2036, 0.1721, 0.2452, 0.2032, 0.1759]",1.298700
1,0.387589,0.062135,"[GOOG, TSLA, AVGO, COST, NFLX]","[0.2818, 0.2236, 0.1998, 0.2726, 0.0224]",1.354314
2,0.612578,0.134769,"[NVDA, NFLX, TSLA, AMD, AMZN]","[0.2373, 0.172, 0.2661, 0.2354, 0.0891]",1.532454
3,0.494391,0.104690,"[AMD, COST, NVDA, META, TSLA]","[0.2351, 0.2184, 0.3637, 0.1323, 0.0506]",1.373446
4,0.429540,0.085512,"[GOOG, NFLX, GOOGL, AVGO, AMD]","[0.1671, 0.2528, 0.1533, 0.2397, 0.1872]",1.297911
...,...,...,...,...,...
995,0.294496,0.075257,"[ADBE, GOOGL, AVGO, META, AMD]","[0.4363, 0.1938, 0.0491, 0.2736, 0.0474]",0.891250
996,0.436016,0.093919,"[TSLA, GOOG, NFLX, AMZN, GOOGL]","[0.2959, 0.2446, 0.1756, 0.1421, 0.1417]",1.259584
997,0.503381,0.114644,"[AMD, NVDA, META, TSLA, AMZN]","[0.1846, 0.333, 0.2386, 0.1094, 0.1344]",1.339023
998,0.381314,0.074842,"[NFLX, NVDA, GOOGL, GOOG, PEP]","[0.2561, 0.2236, 0.3114, 0.1006, 0.1083]",1.211061


Whats next!